In [1]:
!pip install transformers pandas
!pip install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

In [ ]:
import pandas as pd
from transformers import pipeline

df = pd.read_csv('data/Design-Museum-Most-Relevant-360.csv')

# https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment
model_path = 'cardiffnlp/twitter-xlm-roberta-base-sentiment'
sentiment_task = pipeline('sentiment-analysis', model=model_path, tokenizer=model_path)

reviews = df['review_text'].tolist()
results = sentiment_task(reviews)

df['sentiment'] = [result['label'] for result in results]
df['score'] = [result['score'] for result in results]

print(df[['review_text', 'sentiment', 'score']].head())